In [11]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [12]:
df = pd.read_csv("train.csv")
df.shape

(404290, 6)

In [13]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [14]:
new_df = df.sample(30000)

In [15]:
# Duplicated rows
df.duplicated().sum()

np.int64(0)

In [16]:
ques_df = new_df[['question1','question2']]
ques_df.head()

,question1,question2
123748,What conspiracy theories turned out to be true?,What are some conspiracies that turned out to ...
335970,Does mobile service providers record messages?,Will offshore messages be recorded by my mobil...
334739,"How is the general formula for alkanes, alkene...",How is the general formula for alkanes and alk...
304236,What is CADworx all about?,What do you think about this design?
107732,Which is the Best mathematics classes in Delhi?,I am 17 years old can I apply alone fr USA tou...


In [19]:
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
questions = list(ques_df['question1'].fillna('')) + list(ques_df['question2'].fillna(''))

cv = CountVectorizer(max_features=3000)
q1_arr, q2_arr = np.vsplit(cv.fit_transform(questions).toarray(),2)

In [20]:
temp_df1 = pd.DataFrame(q1_arr, index= ques_df.index)
temp_df2 = pd.DataFrame(q2_arr, index= ques_df.index)
temp_df = pd.concat([temp_df1, temp_df2], axis=1)
temp_df.shape

(30000, 6000)

In [21]:
temp_df

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
123748,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
335970,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
334739,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
304236,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
107732,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62376,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
215841,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
68255,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
284706,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
temp_df['is_duplicate'] = new_df['is_duplicate']

In [23]:
temp_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,is_duplicate
123748,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
335970,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
334739,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
304236,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
107732,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(temp_df.iloc[:,0:-1].values,temp_df.iloc[:,-1].values,test_size=0.2,random_state=1)

In [25]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7391666666666666

In [26]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train,y_train)
y_pred = xgb.predict(X_test)
accuracy_score(y_test,y_pred)

0.7286666666666667